In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark=SparkSession.\
    builder.\
        appName("Practice 1").\
            getOrCreate()
            

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/25 11:51:35 INFO SparkEnv: Registering MapOutputTracker
24/05/25 11:51:35 INFO SparkEnv: Registering BlockManagerMaster
24/05/25 11:51:35 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
24/05/25 11:51:35 INFO SparkEnv: Registering OutputCommitCoordinator


In [3]:
flight_df=spark.read.format("csv").\
    option("mode","failfast").\
        option("inferschema","true").\
            option("header","true").\
                load("/user/sovik/retail_test/2010-summary.csv")

In [4]:
flight_df.show(100)

+--------------------+--------------------+------+
|   DEST_COUNTRY_NAME| ORIGIN_COUNTRY_NAME| count|
+--------------------+--------------------+------+
|       United States|             Romania|     1|
|       United States|             Ireland|   264|
|       United States|               India|    69|
|               Egypt|       United States|    24|
|   Equatorial Guinea|       United States|     1|
|       United States|           Singapore|    25|
|       United States|             Grenada|    54|
|          Costa Rica|       United States|   477|
|             Senegal|       United States|    29|
|       United States|    Marshall Islands|    44|
|              Guyana|       United States|    17|
|       United States|        Sint Maarten|    53|
|               Malta|       United States|     1|
|             Bolivia|       United States|    46|
|            Anguilla|       United States|    21|
|Turks and Caicos ...|       United States|   136|
|       United States|         

In [5]:
flight_df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: integer (nullable = true)



In [7]:
flight_df_schema=flight_df.schema

In [9]:
display(flight_df_schema)

StructType([StructField('DEST_COUNTRY_NAME', StringType(), True), StructField('ORIGIN_COUNTRY_NAME', StringType(), True), StructField('count', IntegerType(), True)])

In [15]:
from pyspark.sql.types import StructType,StructField,StringType,IntegerType

In [11]:
list=[(1,"Sovik",35),(2,"Gerona",45),(3,"Sambhu",21)]

In [16]:
schemas=StructType([StructField("ID",IntegerType(),False),
                          StructField("Name",StringType(),False),
                          StructField("Age",IntegerType(),False)])



In [18]:
sample_df=spark.createDataFrame(list,schemas)

In [19]:
sample_df.printSchema()

root
 |-- ID: integer (nullable = false)
 |-- Name: string (nullable = false)
 |-- Age: integer (nullable = false)



In [48]:
new_schema=StructType([StructField("ID",IntegerType(),True),
                          StructField("Name",StringType(),True),
                          StructField("Age",IntegerType(),True),
                          StructField("_corrupt_records",StringType(),True)])

In [37]:
sample_df_new=spark.createDataFrame(sample_df.rdd,new_schema)

In [38]:
sample_df_new.printSchema()

root
 |-- ID: integer (nullable = false)
 |-- Name: string (nullable = false)
 |-- Age: integer (nullable = true)
 |-- __corrupt_fields: string (nullable = true)



In [52]:
permissive_df=spark.read.\
    format("csv").\
            option("inferschema","true").\
            option("header","true").\
            option("mode","PERMISSIVE").\
            schema(new_schema).\
            load("/user/sovik/retail_test/document_v2.csv")

In [53]:
permissive_df.show()

+---+------+----+----------------+
| ID|  Name| Age|_corrupt_records|
+---+------+----+----------------+
|  2|Gerona|  45|            NULL|
|  3|Sambhu|  21|            NULL|
|  4|jerome|NULL|              25|
|  5|  NULL|  25|            NULL|
|  6|Robert|  65|            NULL|
+---+------+----+----------------+



In [54]:
new_schema = StructType([
    StructField("ID", IntegerType(), True),
    StructField("Name", StringType(), True),
    StructField("Age", IntegerType(), True),
    StructField("_corrupt_records", StringType(), True)
])

# Read the CSV file with PERMISSIVE mode
permissive_df = spark.read \
    .format("csv") \
    .option("inferSchema", "true") \
    .option("header", "true") \
    .option("mode", "PERMISSIVE") \
    .schema(new_schema) \
    .load("/user/sovik/retail_test/document_v2.csv")

# Show the DataFrame
permissive_df.show(truncate=False)

# Filter and show only corrupt records
corrupt_records_df = permissive_df.filter(permissive_df["_corrupt_records"].isNotNull())
corrupt_records_df.show(truncate=False)

# Optionally, show valid records as well
valid_records_df = permissive_df.filter(permissive_df["_corrupt_records"].isNull())
valid_records_df.show(truncate=False)

+---+------+----+----------------+
|ID |Name  |Age |_corrupt_records|
+---+------+----+----------------+
|2  |Gerona|45  |NULL            |
|3  |Sambhu|21  |NULL            |
|4  |jerome|NULL|25              |
|5  |NULL  |25  |NULL            |
|6  |Robert|65  |NULL            |
+---+------+----+----------------+

+---+------+----+----------------+
|ID |Name  |Age |_corrupt_records|
+---+------+----+----------------+
|4  |jerome|NULL|25              |
+---+------+----+----------------+

+---+------+---+----------------+
|ID |Name  |Age|_corrupt_records|
+---+------+---+----------------+
|2  |Gerona|45 |NULL            |
|3  |Sambhu|21 |NULL            |
|5  |NULL  |25 |NULL            |
|6  |Robert|65 |NULL            |
+---+------+---+----------------+



In [2]:
spark.stop()